In [ ]:
# add the directory to the path, if not already added
import sys
import os
path_to_add = "/Users/vikra/Downloads/dryrun"  # Replace with the path to your directory
# Ensure the path is absolute
path_to_add = os.path.abspath(path_to_add)
if path_to_add not in sys.path:
  sys.path.append(path_to_add)

# main
import numpy as np
import numpy.linalg as la
import conjgrad
print(dir(conjgrad)) 

rPara=np.array([ [1e-14],      #rPara(1)=10^(-14)=eps
        [1e-9],       #rPara(2)=10^(-9)=epsPivot used in matrix factorization
        [1e-9],       #rPara(3)=10^(-6)=epsSD   10^(-3) is not good
        [0.5*1e-3],   #rPara(4)=0.5*10^(-3), MS9/9/19 relative error of f-value
        [1e-9],       # used in MiniQuadFunUnconstAdapt1.m and MiniQuadFunUnconstAdapt3 , if -df < rPara(5)*|fx0| , not acceptable progress
        [0.5],        # used in MiniQuadFunUnconstAdapt2.m  , if  -gdx*rPara(6) > -df , not acceptable progress  
        [1e18],       # maxDx, max value allowed for a step size in uSD,uPT,uCD
        [1e5],        #rparaFac0=1e+5, since 8/16/23
        [1e-1],       #iparaFac0=1e-1
] )   

iPara=np.array([  [0],        # iPara(1)=option used in LDLtOfSymMatrix21.m  for prearrangement
        [500],       # iPara(2)=max # of iterations for S-D alg
        [0],         # iPara(3)= MS:9/8/19 toDebug
        [21],        # iPara(4)= MS:9/9/19   #-1 is max length of printed out vector/matrix
        [3],         # iPara(5)=0 means use full recursion, iPara(5)=1 means use 1-recursion + steepest descent to solve instead of full recursion
        [0],         # iPara(6)=1 means use SD2 in the framework of v41
        [0],         # iPara(7)= MS:9/15/19 as option to pre-permutation of variables based on domain size
        [1],         # iPara(8)= MS:10/20/19 as option to check suff cond of optimality for earlier exit (1=y, 0=n)
        [0],         # iPara(9)= MS:10/31/19 as option to use 0=LDU (original/default), 1=QDQ' ([Q,D]=eig(A); Q=o.n. e.v. for symmetric case, enough!)
        [0],         # iPara(10)=MS:10/31/19 as option to use different choices of Xstart[]/Ystart[]: 0=Box(original/default), 1=space-Box(hybrid)
        [5],         # iPara(11)=VS:11/7/19, select bdy face 0=original, 1,2,3 for 3 versions of min dist from a line, 4 for min dist from a face approach 11/20
        [0],         # iPara(12)=tie breaker flag to select only one bdy face.
        [0],         # iPara(13)=as a flag controlling a pre-search procedure! It applies to: Sd,AllSd,PsdSd,
] )                

# Example matrix (symmetric for real eigenvalues and eigenvectors)
n=3
A = np.array([[4, 1, -2],          # hessian
                   [1, 2, -1],
                   [-2, -1, 1]])
d = np.array([1, 2, 3])    # linear term
d=d.reshape(-1,1)
x = np.zeros((n,1)) # starting point
#x=np.zeros(n)
diagInv=1/np.diag(A)

# Perform spectral decomposition
diagD, Qmat = la.eigh(A)  # diagD is a vector of eigenvalues in ascending order
                               # Qmat is orthonormal matrix of corresponding eigenvectors
S={}   # create an empty dictionary
neig=sum(diagD>rPara[1])       # S['nNonZeroEig']=sum(diagD>rPara[1])
#print(diagD),print(Qmat[:,-2:])
Q=Qmat[:,n-neig:]                              # S['Q']=Qmat[:,n-S['nNonZeroEig']:]
D=diagD[n-neig:]           # S['D']=diagD[n-S['nNonZeroEig']:]
D=D.reshape(-1,1)
invD=np.array(diagInv[n-neig:])                  #S['diagInv']=np.array(diagInv[n-S['nNonZeroEig']:])

print(Q)  
print(D)
print(neig)
print(invD)

isSoftStop=1
isTF=0
targetF=0
epsx, maxDx, epsg, nMax =rPara[1], rPara[6], rPara[2], iPara[1]

isNew, xnew, fxnew, niters, idExit=conjgrad.cg(n,A,d,Q,D,neig,x,epsx,maxDx,epsg,nMax,isSoftStop,isTF,targetF)
print("isNew:",isNew)
print("xnew:",xnew)
print("fxnew:",fxnew)
print("niters:",niters)
print("idExit:",idExit)
print("xols:" ,-(np.linalg.inv(A)@d))

Next we are building ibbef

In [ ]:
# add the directory to the path, if not already added
import sys
import os
path_to_add = "/Users/vikra/Downloads/dryrun"  # Replace with the path to your directory
# Ensure the path is absolute
path_to_add = os.path.abspath(path_to_add)
if path_to_add not in sys.path:
  sys.path.append(path_to_add)

# main
import numpy as np
from scipy import linalg
from ibbef import main
import conjgrad

rPara=np.array([ [1e-14],      #rPara(1)=10^(-14)=eps
        [1e-9],       #rPara(2)=10^(-9)=epsPivot used in matrix factorization
        [1e-9],       #rPara(3)=10^(-6)=epsSD   10^(-3) is not good
        [0.5*1e-3],   #rPara(4)=0.5*10^(-3), MS9/9/19 relative error of f-value
        [1e-9],       # used in MiniQuadFunUnconstAdapt1.m and MiniQuadFunUnconstAdapt3 , if -df < rPara(5)*|fx0| , not acceptable progress
        [0.5],        # used in MiniQuadFunUnconstAdapt2.m  , if  -gdx*rPara(6) > -df , not acceptable progress  
        [1e18],       # maxDx, max value allowed for a step size in uSD,uPT,uCD
        [1e5],        #rparaFac0=1e+5, since 8/16/23
        [1e-1],       #iparaFac0=1e-1
] )   

iPara=np.array([  [0],        # iPara(1)=option used in LDLtOfSymMatrix21.m  for prearrangement
        [500],       # iPara(2)=max # of iterations for S-D alg
        [0],         # iPara(3)= MS:9/8/19 toDebug
        [21],        # iPara(4)= MS:9/9/19   #-1 is max length of printed out vector/matrix
        [3],         # iPara(5)=0 means use full recursion, iPara(5)=1 means use 1-recursion + steepest descent to solve instead of full recursion
        [0],         # iPara(6)=1 means use SD2 in the framework of v41
        [0],         # iPara(7)= MS:9/15/19 as option to pre-permutation of variables based on domain size
        [1],         # iPara(8)= MS:10/20/19 as option to check suff cond of optimality for earlier exit (1=y, 0=n)
        [0],         # iPara(9)= MS:10/31/19 as option to use 0=LDU (original/default), 1=QDQ' ([Q,D]=eig(A); Q=o.n. e.v. for symmetric case, enough!)
        [0],         # iPara(10)=MS:10/31/19 as option to use different choices of Xstart[]/Ystart[]: 0=Box(original/default), 1=space-Box(hybrid)
        [5],         # iPara(11)=VS:11/7/19, select bdy face 0=original, 1,2,3 for 3 versions of min dist from a line, 4 for min dist from a face approach 11/20
        [0],         # iPara(12)=tie breaker flag to select only one bdy face.
        [0],         # iPara(13)=as a flag controlling a pre-search procedure! It applies to: Sd,AllSd,PsdSd,
] )                

p=5
n=20
k=2
niter=1
for i in range(niter):
    mu=np.zeros(p)
    sigma=np.eye(p)
    seed=np.random.seed(i)
    X = np.random.multivariate_normal(mu, sigma,n )
    colmeans=np.mean(X,axis=0)  # find the mean of each column
    #X= X - colmeans             # make the column mean zero
    colnorms = np.linalg.norm(X, axis=0, keepdims=True) # find 2-norm of each col
    #X = X / colnorms 
    b0 = np.random.randint(-2,2,p)
    b0 = np.reshape(b0, (-1,1)) # make it a col array
    e = np.random.normal(size=n)
    e = np.reshape(e,(-1,1))  # make it a col array
    y = X @ b0 + e
    y = np.reshape(y, (-1,1))  # make it a col array
    A = 2*(X.T @ X)
    b = -2*(X.T @ y)
    c = y.T @ y

    diagInv=1/np.diag(A)

    # Perform spectral decomposition
    diagD, Qmat = linalg.eigh(A)  # diagD is a vector of eigenvalues in ascending order
                               # Qmat is orthonormal matrix of corresponding eigenvectors
    S={}   # create an empty dictionary
    neig=sum(diagD>rPara[1])       # S['nNonZeroEig']=sum(diagD>rPara[1])
    #print(diagD),print(Qmat[:,-2:])
    Q=Qmat[:,p-neig:]                              # S['Q']=Qmat[:,n-S['nNonZeroEig']:]
    D=diagD[p-neig:]           # S['D']=diagD[n-S['nNonZeroEig']:]
    D=D.reshape(-1,1)
    invD=np.array(diagInv[p-neig:])                  #S['diagInv']=np.array(diagInv[n-S['nNonZeroEig']:])

    #elt=Q@(D*Q.T)==A
    #print(elt)
    #print(Q@(D*Q.T))
    #print(A)
      

    #print(Q)  
    #print(D)
    print('neig:',neig)
    #print(invD)

    isSoftStop=1
    isTF=0
    targetF=0
    epsx, maxDx, epsg, nMax =rPara[1], rPara[6], rPara[2], iPara[1]

    R, ipiv0 = linalg.qr(A, mode='r', pivoting=True)
    diagR=np.abs(np.diag(R))

    #print("Q:",Q)
    #print("R:",R)
    #print("P:",ipiv0)
    #print("diagR:",diagR)
    x0=np.zeros((p,1))
    #print('X,e,y,b0:',X,e,y,b0)
    main(p,n,y,X,A,Q,D,b,c,k,x0)
    #xols=-(np.linalg.inv(A)@b)
    #print("xols:" ,xols)
    #print('fxols:',0.5*(xols.T@A@xols)+b.T@xols+c)
    #print('fxols:',b.T@xols)
    x0=np.zeros((p,1))
    isNew, xnew, fxnew, niters, idExit=conjgrad.cg(p,A,b,Q,D,neig,x0,epsx,maxDx,epsg,nMax,isSoftStop,isTF,targetF)
    #print("isNew:",isNew)
    print("xnew:",xnew)
    print("fxnew:",fxnew+c)
    #print("niters:",niters)
    #print("idExit:",idExit)

neig: 5
A: [[44.77013425  6.01575924  7.89814632  6.43055005 -8.68614803]
 [ 6.01575924 36.98643427  9.37834844 -1.3669892  10.77466413]
 [ 7.89814632  9.37834844 28.95427158 -3.81197191 12.24223124]
 [ 6.43055005 -1.3669892  -3.81197191 60.45381249  2.06314886]
 [-8.68614803 10.77466413 12.24223124  2.06314886 32.71607101]]
ipiv0: [3 0 1 4 2]
ipiv1: []
A: [[60.45381249  6.43055005 -1.3669892   2.06314886 -3.81197191]
 [ 6.43055005 44.77013425  6.01575924 -8.68614803  7.89814632]
 [-1.3669892   6.01575924 36.98643427 10.77466413  9.37834844]
 [ 2.06314886 -8.68614803 10.77466413 32.71607101 12.24223124]
 [-3.81197191  7.89814632  9.37834844 12.24223124 28.95427158]]
b: [[-56.06959023]
 [ 62.42410182]
 [ 93.12838595]
 [ 12.09748502]
 [ 15.16278545]]
npiv0, npiv00: 5 5
E0: [[ 1.          0.10637129 -0.02261213  0.03412769 -0.06305594  0.92747815]
 [ 0.          1.          0.13975304 -0.20200485  0.18835028 -1.55124367]
 [ 0.          0.          1.          0.33428661  0.22528912 -2.280

In [5]:
import numpy as np
a=np.ones(5,dtype=np.int32)
b=(1,a[2]+1)
print('b:',b[1]=3)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (3024600541.py, line 4)